# Create an auto testing script for Qwen2.5-14B-Instruct

In [13]:
import requests
import copy
import json
import time
import csv
import pandas as pd

In [44]:
url = 'https://aihub.vietteltelecom.vn:8443/api/llm/test-prompt'
headers = {
    'Content-Type': 'application/json',
    'x-api-key': 'CjuiGkZSLSyxWspiQaRfiA',
}

In [47]:
template = {'max_tokens': 100,
            'temperature': 0.0,
            'stream': False,
            'model': 'Qwen2.5-14B-Instruct',
            'messages': [{'role': 'system', 'content': ''},
                         {'role': 'user', 'content': ''}]}

In [50]:
# ROUTING_PROMPT = open('ROUTING_PROMPT_AIBOX.txt', 'r', encoding='utf-8').read()
# COMMON_EXTRACT_PROMPT = open('COMMON_EXTRACT_PROMPT.txt', 'r', encoding='utf-8').read()
# CHANNEL_EXTRACT_PROMPT = open('CHANNEL_EXTRACT_PROMPT.txt', 'r', encoding='utf-8').read()
path = '/Users/percyd/Docs/ViettelDAC_AI/LlmPromptingProject/'
ROUTING_PROMPT = open(path+'./prompts/ROUTING_PROMPT_AIBOX.txt', encoding='utf-8').read()

In [14]:
test_df = pd.read_csv(path+'./tests/input_test_intent.csv', encoding='utf-8')

In [45]:
def send_request(url_route, header, data, prompt_type, usr_in):
    temp_data = copy.deepcopy(data)
    temp_data['messages'][0]['content'] = prompt_type
    temp_data['messages'][1]['content'] = usr_in
    print(temp_data)
    try:
        start = time.time()
        response = requests.post(url=url_route, headers=header, json=temp_data)
        duration = time.time() - start
        print(response)
        response.raise_for_status()
        status_code = response.status_code
        print(f'Status Code: {status_code}')
        if status_code == 200:
            res = response.json()
            print(f'Response JSON: {res}')
            print(f'Time respond: {duration*1000:.0f}ms')
            return json.loads(res).get('intent'), duration
        else:
            print(f'Response Text: {response.text}')
            return None, 0
    except requests.exceptions.RequestException as e:
        print(f'An error occurred: {e}')
        return None, 0

In [17]:
converter = {
    "cmd_raise_volume": "volume_control",
    "cmd_reduce_volume": "volume_control",
    "cmd_max_volume": "volume_control",
    "cmd_min_volume": "volume_control",
    "cmd_volume_on": "volume_control",
    "cmd_volume_off": "volume_control",
    "cmd_pause": "playback_control",
    "cmd_continues": "playback_control",
    "cmd_fast_forward": "playback_control",
    "cmd_rewind": "playback_control",
    "cmd_turn_on": "device_control",
    "cmd_turn_off": "device_control",
    "navigate_screen": "navigate_screen",
    "navigate_app": "navigate_app",
    "open_item": "content_open",
    "opem_program": "content_open",
    "open_schedule": "open_schedule",
    "cmd_search": "cmd_search",
    "film_qna": "search_item",
    "general_query": "search_item",
    "gold_price_query": "search_item",
    "weather_query": "search_item",
    "package_query": "search_item"
}

In [32]:
test_df['coarse_intent'] = test_df['expected_intent'].map(converter).fillna('unknown')

In [51]:
send_request(url,headers,template,ROUTING_PROMPT,'mở vtv1')

{'max_tokens': 100, 'temperature': 0.0, 'stream': False, 'model': 'Qwen2.5-14B-Instruct', 'messages': [{'role': 'system', 'content': 'Bạn là trợ lý phân loại yêu cầu người dùng trong ứng dụng xem nội dung trực tuyến. Nhiệm vụ của bạn là chọn hành động tiếp theo dựa trên input của người dùng, chỉ trả về duy nhất MỘT hành động phù hợp nhất. KHÔNG trả ra cái gì khác (kể cả Lưu ý) ngoài output format.\n\n---\n\nTrả về đúng JSON định dạng:\n{"intent": "Tên hành động phù hợp"}\nKHÔNG được trả thêm nội dung nào khác.\n\n---\n\n*intent* là một trong các hành động sau:\n- open_program: Mở **chương trình truyền hình cụ thể** (ví dụ: "Thời sự", "Rap Việt",...), khi người dùng **muốn mở chương trình**. KHÔNG dùng nếu câu chứa tên **một phim, kênh hoặc video cụ thể** → chuyển sang `open_item`.KHÔNG được nhầm lẫn với KÊNH TRUYỀN HÌNH. KHÔNG nhầm lẫn sang `open_schedule` khi có ngày giờ cụ thể.\n- navigate_screen: Điều hướng tới màn hình khác. Trợ lý này có thể điều hướng tới các trang / màn hình: Tr

TypeError: the JSON object must be str, bytes or bytearray, not dict

In [26]:
test_df.drop(columns=['Unnamed: 0'], inplace=True)

In [29]:
test_df[['pred_coarse_intent', 'time']] = test_df['user_input'].apply(
    lambda x: send_request(url,
                           headers,
                           template,
                           ROUTING_PROMPT,
                           x) if isinstance(x, str) else (None, 0)
).apply(pd.Series)


Status Code: 200
An error occurred: Expecting value: line 1 column 1 (char 0)
Status Code: 200
An error occurred: Expecting value: line 1 column 1 (char 0)
Status Code: 200
An error occurred: Expecting value: line 1 column 1 (char 0)
Status Code: 200
An error occurred: Expecting value: line 1 column 1 (char 0)
Status Code: 200
An error occurred: Expecting value: line 1 column 1 (char 0)
Status Code: 200
An error occurred: Expecting value: line 1 column 1 (char 0)
Status Code: 200
An error occurred: Expecting value: line 1 column 1 (char 0)
Status Code: 200
An error occurred: Expecting value: line 1 column 1 (char 0)
Status Code: 200
An error occurred: Expecting value: line 1 column 1 (char 0)
Status Code: 200
An error occurred: Expecting value: line 1 column 1 (char 0)


KeyboardInterrupt: 

In [9]:
df['name'].to_csv('ctth.txt', index=False, header=False)

In [11]:
import requests

# Kiểm tra xem requests đang dùng cái nào
try:
    import charset_normalizer
    print("requests đang sử dụng charset_normalizer:", charset_normalizer.__version__)
except ImportError:
    try:
        import chardet
        print("requests đang sử dụng chardet:", chardet.__version__)
    except ImportError:
        print("Không tìm thấy cả chardet lẫn charset_normalizer.")

Không tìm thấy cả chardet lẫn charset_normalizer.


In [12]:
print("requests version:", requests.__version__)

requests version: 2.32.4


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "Qwen/Qwen2.5-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk.


In [2]:
def run_qwen_chat(prompt: dict) -> str:
    messages = prompt.get("messages", [])
    temperature = prompt.get("temperature", 0.0)
    max_tokens = prompt.get("max_tokens", 256)
    top_k = prompt.get('top_k', 50)
    top_p = prompt.get('top_p', 0.9)
    repetition_penalty = prompt.get('repetition_penalty', 1.0)

    # Create prompt from chat messages
    chat_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # Tokenize and generate
    inputs = tokenizer(chat_prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=temperature,
        do_sample=temperature > 0
    )

    # Decode and extract generated response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split(chat_prompt)[-1].strip()

In [5]:
result = run_qwen_chat(prompt)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [4]:
prompt = {
    'top_k': 0.0,
    'top_p': 0.0,
    'repetition_penalty': 0.0,
    'max_tokens': 1024,
    'temperature': 0.0,
    'stream': False,
    'model': 'Qwen2.5-3B-Instruct',
    'messages': [
        {
            'role': 'system',
            'content': 'Bạn là hệ thống phân loại yêu cầu người dùng trong ứng dụng xem nội dung.\n\nNhiệm vụ:\n- Dựa trên câu lệnh người dùng, chọn **DUY NHẤT MỘT nhóm hành động** phù hợp nhất.\n- Trả về đúng định dạng JSON: {"group": "Tên nhóm"}\n- KHÔNG trả thêm nội dung nào khác.\n\nDanh sách nhóm:\n\n- "media_control": Điều khiển phát lại, âm lượng, tua, tạm dừng, tiếp tục, bật/tắt thiết bị.\n- "content_open": Mở hoặc hỏi về nội dung: phim, chương trình, video, lịch chiếu.\n- "query_info": Câu hỏi thông tin, phản ánh, tìm kiếm, hỏi về phim, gói cước, thời tiết, vàng.\n- "app_ui": Điều hướng giao diện ứng dụng, mở YouTube, quay lại trang trước, tắt màn hình.\n\nVí dụ:\n- "Tăng âm lượng lên" → {"group": "media_control"}\n- "Xem phim Avatar" → {"group": "content_open"}\n- "Không thấy phim yêu thích" → {"group": "query_info"}\n- "Về trang chủ giúp tôi" → {"group": "app_ui"}'
        },
        {
            'role': 'user',
            'content': 'tôi muốn xem phim hoạt hình'

        }
    ]
}